Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

**Deadline**

T.b.d.


In [152]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
#import numpy as np

## The *Nim* and *Nimply* classes

In [153]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [154]:
# The k can be omitted, it isn't always constrained the max number of object that can be taken at once (in one move)
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    # this allow to use the while loop on the object (when all rows are zeroed this returns false)
    def __bool__(self):
        return sum(self._rows) > 0

    # representation "override"
    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)
    
    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [155]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [156]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [157]:
def nim_sum(state: Nim) -> int:
    result = state.rows[0]
    for row in state.rows[1:]:
        result = result ^ row
    return result
    
    #tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    #xor = tmp.sum(axis=0) % 2
    #return int("".join(str(_) for _ in xor), base=2)

def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked['possible_moves'] = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k]
    cooked['active_rows_number'] = sum(o > 0 for o in state.rows)
    cooked['shortest_row'] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y:y[1])[0]
    cooked['longest_row'] = max((x for x in enumerate(state.rows)), key=lambda y:y[1])[0]
    cooked['nim_sum'] = nim_sum(state)

    brute_force = list()
    for m in cooked['possible_moves']:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked['brute_force'] = brute_force

    return cooked

In [158]:
def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    #return next(m for m in data['possible_moves'] if m[1] == data['min_sum'])
    return next((bf for bf in data['brute_force'] if bf[1] == 0), random.choice(data['brute_force']))[0]

In [159]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome['p']:
            ply = Nimply(data['shortest_row'], random.randint(1, state.rows[data['shortest_row']]))
        else:
            ply = Nimply(data['longest_row'], random.randint(1, state.rows[data['longest_row']]))

        return ply
    return evolvable

## Evaluation of a strategy

In [160]:
NUM_MATCHES = 100
NIM_SIZE=10

def evaluate(strategy: Callable) -> float:
    #opponent = (strategy, pure_random)
    opponent = (strategy, optimal_strategy)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won/NUM_MATCHES


## Testing

In [161]:
evaluate(make_strategy({'p' : .1}))

0.47

## (OLD) Oversimplified match

In [162]:
logging.getLogger().setLevel(logging.DEBUG)

#strategy = (pure_random, gabriele)
strategy = (make_strategy({'p' : .1}), optimal_strategy)

# 11 is the num of rows
nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
turn = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    turn = turn+1
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won! (At turn {turn})")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 19 18>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 11 13 15 1 19 18>
DEBUG:root:status: After player 0 -> <0 3 5 7 9 11 13 15 1 19 18>
DEBUG:root:status: After player 1 -> <0 2 5 7 9 11 13 15 1 19 18>
DEBUG:root:status: After player 0 -> <0 2 5 7 9 11 13 15 1 1 18>
DEBUG:root:status: After player 1 -> <0 2 5 7 9 11 13 15 1 1 0>
DEBUG:root:status: After player 0 -> <0 2 5 7 9 11 13 12 1 1 0>
DEBUG:root:status: After player 1 -> <0 1 5 7 9 11 13 12 1 1 0>
DEBUG:root:status: After player 0 -> <0 0 5 7 9 11 13 12 1 1 0>
DEBUG:root:status: After player 1 -> <0 0 4 7 9 11 13 12 1 1 0>
DEBUG:root:status: After player 0 -> <0 0 4 7 9 11 1 12 1 1 0>
DEBUG:root:status: After player 1 -> <0 0 4 7 5 11 1 12 1 1 0>
DEBUG:root:status: After player 0 -> <0 0 4 7 5 11 1 4 1 1 0>
DEBUG:root:status: After player 1 -> <0 0 4 7 5 3 1 4 1 1 0>
DEBUG:root:status: After player 0 